In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
using Gridap
using GridapGmsh
using Gridap.Geometry

In [ ]:
function thermistor(msh, U₀, fname, Tw = 200.0, order=2, Tref=0.0)
    Prₐ = 0.707
    kₐ = 25.3e-3
    νₐ = 1.6e-5

    D₁ = 2e-3
    D₂ = 0.5e-3
    Re₁ = U₀ * D₁ / νₐ
    Re₂ = U₀ * D₂ / νₐ
    println(Re₁)
    Nu₁ = 0.51 * Re₁^0.5 * Prₐ^0.37
    Nu₂ = 0.51 * Re₂^0.5 * Prₐ^0.37

    k₁ = 1.0
    k₂ = 50.0
    h₁ = Nu₁ / D₁ * kₐ/k₁
    h₂ = Nu₂ / D₂ * kₐ/k₂
    println(h₁)
    model = GmshDiscreteModel(msh)
    degree = 2*order
    reffe = ReferenceFE(lagrangian, Float64, order)
    V = TestFESpace(model, reffe, dirichlet_tags="temp")
    U = TrialFESpace(V, Tw-Tref)
    println(Tw-Tref)
    Ω = Triangulation(model)
    dΩ = Measure(Ω, degree)    

    Γ₁ = BoundaryTriangulation(model, tags="hglass")
    dΓ₁ = Measure(Γ₁, degree)    
    
    Γ₂ = BoundaryTriangulation(model, tags="hsteel")
    dΓ₂ = Measure(Γ₁, degree)    
    
    labels = get_face_labeling(model)
    dimension = 2
    tags = get_face_tag(labels, dimension);    
    tag_steel = get_tag_from_name(labels, "steel")
    tag_glass = get_tag_from_name(labels, "glass")    
    
    heatflux = (∇u, tag) -> begin
        if tag == tag_steel
            return h₂ * ∇u
        else
            return h₁ * ∇u
        end 
    end
    
    f(x) = 0.0
    a(u,v) = ∫( ∇(v)⋅ (heatflux∘(∇(u), tags) ) )*dΩ + ∫(v*u*h₁)*dΓ₁ + ∫(v*u*h₂)*dΓ₂
    b(v) = ∫(v*f)*dΩ
    
    op = AffineFEOperator(a,b,U,V)
    
    ls = LUSolver()
    solver = LinearFESolver(ls)
    
    uh = solve(solver, op)
    
    writevtk(Ω, fname, cellfields=["T"=>uh+Tref])
    return uh
end


In [ ]:
numstring(x, n=3) = string(x + 10^n)[2:end]

In [ ]:
heatflux

In [ ]:
U = [1.0, 5.0, 10.0, 20.0] #[0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4, 5, 6, 8, 10, 12, 15, 20]
fnames = "thermistor_" .* numstring.(round.(Int, 10*U))
thermistor.("mf58.msh", U, fnames)

In [ ]:
0.51 * 10 * 0.7^0.37

In [ ]:
u1 = thermistor("mf58.msh", 1.0, "lixo_01")

In [ ]:
u2 = thermistor("mf58.msh", 10.0, "lixo_10")

In [ ]:
u2()

In [ ]:
u2(0.5e-3, 0.5e-3)